### 3. Kommunikation mit dem Netzwerk - Teil 1: Basics

Um Informationen (im Wesentlichen Blöcke und Transaktionen) miteinander auszutauschen, müssen FullNodes untereinander und mit Wallets kommunizieren. In unserem Fall wird die Kommunikation über eine Art Web-Interface ermöglicht. Wir benutzen das *Microframework* Flask um die Kommunikation zu ermöglichen. Was ein *Microframework* ist, wird [hier](https://en.wikipedia.org/wiki/Microframework) genauer beschrieben. Ein ausführliches Tutorial zu Flask gibt es  [hier](https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-i-hello-world).  
Uns bietet Flask in erster Linie eine Möglichkeit, von einem FullNode A aus (bzw. einer Wallet A aus) bestimmte Funktionen des FullNodes B aufzurufen oder umgekehrt (im Fall der Wallet nur einseitige Kommunikation mögich). Z.B. können wir per Flask implementieren, dass FullNode B bei seiner Initialisierung eine Methode `get_list_of_seen_blocks` von FullNode A aufrufen kann, um dadurch eine Liste aller Blöcke, die FullNode A bekannt sind, zu erhalten. Durch den Aufruf einer weiteren Methode `get_block_by_id` von FullNode A kann FullNode B dann die gesamten Blöcke, die ihm selbst noch nicht bekannt sind, bei FullNode A anfragen.  
Anhand des folgenden Beispiels soll die Funktionsweise von Flask verdeutlicht werden.

#### Flask Beispiel

Um einen Überblick über das Vermögen eines Public Keys zu haben, muss eine Wallet an einen FullNode angebunden werden. Zu diesem Zweck verwenden wir seitens der `FullNode` Klasse die folgenden Methode:

```python

## New (4_2) 1
def get_UTXOs_for_public_key(self, public_key):
    # Returns an OD of unspent UTXOs of the primary chain 
    # which are also not part of mempool transactions 
    # (of the primary chain's mempool)
    
    UTXOs_for_public_key = OrderedDict()
    # Loop through UTXO ids
    for UTXO_id in self.chains[str(self.primary_chain_id)].UTXOs.keys():
        # Get rid of the UTXOs used in another 
        # mempool transaction:
        if UTXO_id  not in \
        self.chains[str(self.primary_chain_id)].mempool_UTXOs.keys():
            # Get the UTXO for the specific id
            candidate_UTXO = self.chains[str(self.primary_chain_id)].\
            get_UTXO_by_id(UTXO_id)

            # If UTXO belongs to public key, add it to return dict.
            if candidate_UTXO.recipient == public_key:
                UTXOs_for_public_key[candidate_UTXO.id] = \
                candidate_UTXO.get_full_odict()
    return(UTXOs_for_public_key)
```

Die Methode itteriert über alle *UTXOs* der aktuellen *Primary-Chain* des FullNodes und gibt die UTXOs zurück, die im Besitz des `public_keys` sind. D.h. eine Wallet kann mit Hilfe dieser Methode herausfinden welches Vermögen aktuell zu einem bestimmten `public_key` gehört. Die `get_UTXOs_for_public_key` Methode ist der `FullNode` Klasse in diesem Verzeichniss bereits hinzugefügt.
Um die Methode nun extern aufzurufen (extern bedeutet hier nicht zwangsläufig von einem anderen Computer, sondern vielmehr von außerhalb des Python Programms in dem sie implementiert wird, d.h. von außerhalb der entsprechenden FullNode Instanz) starten wir eine Flask Applikation innerhalb der die FullNode Instanz initialisiert wird. Das wird mit Hilfe der `run_full_node.py` Datei gemacht, die in diesem Verzeichniss liegt. Der Inhalt der Datei sieht folgendermaßen aus - die Kommentare erläutern die einzelnen Programmabschnitte konkret.  
Versuche den Code soweit es geht zu verstehen. Eventuelle Verständnisslücken können möglicherweise auch durch die anschließende Ausführung klar werden.

```python

from flask import Flask, request
from FEDCoin import *
import json

# Creating an instance of the FullNode class. 
# This is THE FullNode in this example.
full_node = FullNode()

# Starting Flask
app = Flask(__name__)

# This is the anchor to call 'get_UTXOs_for_public_key'
# for any external client.
# "/get_UTXOs_for_public_key" defines the path,
# methods=['GET'] defines by which request type this anchor can 
# be called.
# Note that request.form['public_key'] is a variable passed 
# within the request. 
# The 200 in the return statement just indicates the client
# that everything went fine 
# (i.e. 404 is the code for a well known error you commonly get when browsing the web).
@app.route("/get_UTXOs_for_public_key", methods=['GET'])
def get_UTXOs_for_public_key():
    # This is the line where the full_node 
    # instance calls its method get_UTXOs_for_public_key
    # with the request.form['public_key'] argument.
    response =\
    full_node.get_UTXOs_for_public_key(request.form['public_key'])
    # Return the response in json format
    return json.dumps(response), 200
    
# This is a test anchor for hello world
@app.route("/hello_world", methods=['GET'])
def hello_world():
    # Prints hello world to the console and returns
    # hello world.
    # Just for testing!
    print('hello-world')
    return('hello world!')


# This if clause is entered if run_full_node.py 
# is executed via the console. In this case the __name __ variable
# is automatically set equal to '__main__'.
# The program can be called via the console using:
# run_full_node.py -p 5001
# If no port is surpassed, the default port 5000 is used.
if __name__ == '__main__':
    from argparse import ArgumentParser

    parser = ArgumentParser()
    parser.add_argument('-p', '--port', default=5000, type=int, \
        help='Listening port for the FullNode')

    # Parse the arguments that are passed when the 
    # run_full_node_example.py is started via the console
    args = parser.parse_args()
    port = args.port

    # Run the Flask App
    app.run(host='127.0.0.1', port=port)


```

Um die `run_full_node.py` Datei auszuführen und die Flask Applikation samt FullNode Instanz zu starten, öffne eine Konsole und navigiere in den ensprechenden Ordner. Aktiviere das `fed_coin` Environment (MacOS: `source activate fed_coin` oder Windows: `activate fed_coin` und gib folgenden Befehl in die Konsole ein:

`python run_full_node.py -p 5001`

Dadurch wird die Flaks Applikation lokal gestartet und der FullNode läuft im Hintegrund. Nach erfolgreichem Start sollte in der Konsole Folgendes stehen:  

`* Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)`

Öffne nun eine Browserfenster und rufe folgende Adresse auf:

`http://127.0.0.1:5001/hello_world`

Daraufhin sollte sich eine Seite mit 'hello world!' als einzigem Inhalt im Browser öffnen. Das geschieht weil wir als Rückgabewert `return(hello world!)` im Code angegeben haben. Gleichzeitig sollte in der Konsole, mit der das Programm gestartet wurde, `hello-world` ausgegeben werden. Diese Ausgabe ist auf die Codezeile `print('hello-world')` zurückzuführen. Die Konsole kannst Du danach einfach minimieren (unser Flask Server läuft dann im Hintegrund weiter). 

#### Erweiterung der Wallet Klasse (1)

Um nun die gewünschte Funktionalität in *Wallet* einzubauen, erweitern wir die `Wallet` Klasse so, dass eine *Wallet* stets die UTXOs, die mit dem `public_key` der Wallet gehören bei einem FullNode abfragen kann. Basieren darauf kann eine *Wallet*-Instanz dann ihren aktuellen Konstostand berechnen. Die erweiterte Implementierung der `Wallet` Klasse sieht folgendermaßen aus:

```python
class Wallet:
    ## New (4_2) 2 -- Parameters
    def __init__(self, private_key_str = None, parent_node_address = None):
        keypair = self.generate_keypair(private_key_str)
        self.private_key = keypair['private_key']
        self.public_key = keypair['public_key']
        
        ## New (4_2) 3 -- start ##
        # UTXOs owned by the wallet; 
        # format: TransactionOutput.id:TransactionOutput
        self.UTXOs = OrderedDict({}) 
        self.parent_node = None # String i.e. '127.0.0.1:5001'
        self.balance = None # Floating Value of the balance
        # Note that the following lines have to be at the
        # end of the constructor.
        if parent_node_address is not None:
            self.parent_node = parent_node_address
            self.calculate_balance()
        ## New (4_2) 3  -- end ##


    def generate_keypair(self,private_key_str):
        # If this is a 'new' wallet, i.e. 
        # no private_key_str has been given to the constructor
        # --> Create a private key
        if private_key_str is None:
            # Generate the private key 
            # (in here the randomness takes place)
            private_key = ECC.generate(curve='P-256')
        else:
            # In case this is a wallet for a specific private_key
            # read the private key in string format.
            # binascii.unhexlify(x) converts x which is in 
            # hexadecimal format into a binary format
            private_key = ECC.import_key(binascii.unhexlify(private_key_str))

        # We return a dictionary with private and public key
        keypair = {
        'private_key':\
        binascii.hexlify(private_key.export_key(format='DER'))\
        .decode('utf8'),
        'public_key':\
        binascii.hexlify(private_key.public_key()\
        .export_key(format='DER')).decode('utf8')
        }
        return keypair

    ## New (4_2) 4
    def update_UTXOs(self):
        request_data = {'public_key' : self.public_key}
        UTXOs_json = requests.get('http://' + self.parent_node  \
        + '/get_UTXOs_for_public_key', data = request_data).content

        # Clear old UTXOs
        self.UTXOs = OrderedDict({})
        # Loop through the result and generate TransactionOutput objects
        # Add them to the UTXOs ordered dict
        for UTXO_id, UTXO_odict in create_odict_from_json(UTXOs_json).items():
            self.UTXOs[UTXO_id] = TransactionOutput.from_odict(UTXO_odict)

    ## New (4_2) 5
    def calculate_balance(self):
        if self.parent_node is not None:
            self.update_UTXOs()
            balance = 0
            for UTXO in self.UTXOs.values():
                balance += UTXO.value

            self.balance = balance
   
```

Die Klasse wird hierfür an den im Code markierten Stellen (`## New (4_2) 2...` bis `## New (4_2) 5`) erweitert. Hier die Erläuterungen zu den verschiedenen Stellen:

* **## New (4_2) 2**: Als zusätzlicher Parameter kann der Wallet bei Initialisierung nun die Adresse eines *Parent Nodes* übergeben werden. Die Adresse soll im Format '127.0.0.1:5000' sein.
<br/>
<br/>
* **## New (4_2) 3**: Jedes Objekt der Klasse wird nun zusätzlich mit einem `OrderedDict` namens UTXOs initialisiert. Hier werden alle UTXOs abgespeichert, über die die entsprechende Wallet verfügen kann. Außerdem hat jedes Wallet Objekt zusätzlich die Eigenschaften `parent_node` (die Adresse des Parent *FullNodes* als String) und `balance` (Wert der UTXOs aus dem `UTXOs` `OrderedDict`). Der Code am Ende des Konstruktors wird nur ausgeführt, wenn der Wallet bei Initialisierung eine `parent_node_address` übergeben wurde. Ansonsten kann keine `balance` berechnet werden.
<br/>
<br/>
* **## New (4_2) 4**: Die Funktion `update_UTXOs` führt die Kommunikation mit der Flask Applikation (d.h. dem *FullNode*) durch. Wir verwenden hierzu das `requests` package in Python. Mit Hilfe des `requests` packages können wir den vorher manuell durchgeführten Aufruf der Webadresse in Python realisieren. Durch die `requests` können außerdem auch Parameter übergeben werden. Wir übergeben hier den Wert des *Public Keys* der entsprechenden Wallet mit Hilfe eines `Dictionaries`. Für den Aufruf verwenden wir einen sogenannten `get` request, weil wir Daten vom *FullNode* bekommen möchten. Das `request_data` `Dictionary` wird beim Aufruf übergeben, als Antwort bekommen wir ein `json` Objekt. Dieses `json` Objekt wandeln wir mit Hilfe der folgenden Funktion in ein `OrderedDict` um:
  ```python
  
  def create_odict_from_json(json_stuff):
    return json.loads(json_stuff.decode('utf8'), object_pairs_hook=OrderedDict)
  ```
  Die `create_odict_from_json` Methode ist bereits dem `utils` Modul das in diesem Verzeichiss verwendet wird hinzugefügt.
  Innerhalb des `for` Loops in der `update_UTXOs` Methode der `Wallet` Klasse werden dann alle UTXOs in ein `TransactionOutput` Objekt umgewandelt und dem `self.UTXOs` `OrderedDict` hinzugefügt. Um aus einem `OrderedDict` Objekt direkt ein `TransactionOutput` Objekt zu erzeugen, verwenden wir die `TransactionOutput.from_odict()` Funktion. Das ist eine sogeannte *Klassen Funktion*, die bei uns folgendermaßen definiert ist (als Teil der `TransactionOutput` Klasse, die in diesem Verzeichniss hinterlegt ist):
  ```python
    ## New (4_2) 6
    @classmethod
    def from_odict(cls, transaction_output_odict):
        #transaction_output_odict = create_odict_from_json(transaction_output_json)
        response = cls(transaction_output_odict['recipient'],
                        transaction_output_odict['value'],
                        transaction_output_odict['random'],
                        transaction_output_odict['id'],
                        transaction_output_odict['time'],)
        return response
  ```
  Das wesentliche Merkmal einer Klassenfunktion ist, dass als erster Parameter immer `cls` übergeben wird. `cls` ist ein Platzhalter für den enstprechenden Klassennamen in der die Methode implementiert wird. In unserem Fall hier ist `cls` = `TransactionOutput` (das läuft im Hintegrund ab), weil wir die Methode innerhalb der `TransactionOutput` Klasse implementieren. Durch den Aufruf von `cls(..)` wir dann einfach der `TransactionOutput` Konstruktor aufgerufen. Die Methode vereinfacht die Erzeugung einer `TransactionOutput` Instanz basierend auf einem `OrderedDict`. Weil *Flask* über *json* kommunizieren kann und *json* einfach in ein `OrderedDict` umgewandelt werden kann, verwenden wir dieses 'Werkzeug'. Zusätzlich zur Klassenmethode in der `TransactionOutput` Klasse fügen wir zwei gelichnamige (ebenfalls `from_odict`) Klassemnmethoden auch für die Klassen `Transaction` (`# New (4_2) 7`) und `Block` (`# New (4_2) 8`) ein, die wir aber erst später benötigen. Suche in der `FEDCoin.py` Datei in diesem Verzeichnis nach `@classmethod` um Dir die genaue Implementierung der Methoden anzuschauen. Sie sind alle nach dem gleichen Muster aufgebaut.
<br/>
<br/>
* **## New (4_2) 5**: Die Funktion `calculate_balance` ruft zuerst die `update_UTXOs` Methode auf um dann im Anschluss daran die Summe der in den UTXOs hinterlegten `value`s zu bilden.
  
Wenn wir nun eine Wallet mit dem Genesis *Private Key* erzeugen, können wir die Balance der Genesis Wallet basierend auf der *Primary Chain* des mit Hilfe von Flask gestarteten FullNodes abfragen. Der Genesis *Private Key* lautet: `308187020100301306072a8648ce3d020106082a8648ce3d030107046d306b020101042012bbcc17335fa7345f7be65b01a5d6dca706d8f2ba99691f314697f288d678c0a144034200043f3bd6d16ce4bde95a8237170aaa106388485498234a3dca5d0c273d907c03c3e72017bec13cf6e893e96da0f9d6c7037c79b40cb006aff12ae88adae1b0bb7e`

In [ ]:
# Automatically relaoad modules that have changed while
# being loaded.
%load_ext autoreload
# Set equal to 2 to enable auto realod,
# set equal to 0 to disable auto relaod.
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Import stuff
# Import the modules we need
from IPython.core.debugger import set_trace

from FEDCoin import *
from utils import *

In [5]:
gen_priv_key = '308187020100301306072a8648ce3d020106082a8648ce3d030107046d306b020101042012bbcc17335fa7345f7be65b01a5d6dca706d8f2ba99691f314697f288d678c0a144034200043f3bd6d16ce4bde95a8237170aaa106388485498234a3dca5d0c273d907c03c3e72017bec13cf6e893e96da0f9d6c7037c79b40cb006aff12ae88adae1b0bb7e'
parent_address = '127.0.0.1:5001'
gen_wallet = Wallet(gen_priv_key, parent_address)

# Check the balance (should output 50):
print(gen_wallet.balance)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5001): Max retries exceeded with url: /get_UTXOs_for_public_key (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1060925f8>: Failed to establish a new connection: [Errno 61] Connection refused',))